In [ ]:
!nvidia-smi

Thu Aug 18 16:35:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd 
%cd drive/MyDrive/models/dino-main/
!nvidia-smi
!pip install timm==0.4.9
!pip install yacs
!pip install -U PyYAML
!pip install einops

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
/content/drive/MyDrive/models/dino-main
Fri Aug 19 17:01:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |              

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path
from dataset import SingleData
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
from eval_knn import extract_features
import utils
import vision_transformer as vits
from vision_transformer import DINOHead
from numpy import load

def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index,self.img_list[index]


KNN=load('/content/drive/MyDrive/models/dino-main/KNN2.npy')
torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))

def get_args_parser():
    parser = argparse.ArgumentParser('DINO', add_help=False)

    # Model parameters
    parser.add_argument('--arch', default='vit_base', type=str,
        choices=['vit_tiny', 'vit_small', 'vit_base', 'xcit', 'deit_tiny', 'deit_small'] \
                + torchvision_archs + torch.hub.list("facebookresearch/xcit:main"),
        help="""Name of architecture to train. For quick experiments with ViTs,
        we recommend using vit_tiny or vit_small.""")
    parser.add_argument('--patch_size', default=16, type=int, help="""Size in pixels
        of input square patches - default 16 (for 16x16 patches). Using smaller
        values leads to better performance but requires more memory. Applies only
        for ViTs (vit_tiny, vit_small and vit_base). If <16, we recommend disabling
        mixed precision training (--use_fp16 false) to avoid unstabilities.""")
    parser.add_argument('--out_dim', default=512, type=int, help="""Dimensionality of
        the DINO head output. For complex and large datasets large values (like 65k) work well.""")
    parser.add_argument('--norm_last_layer', default=True, type=utils.bool_flag,
        help="""Whether or not to weight normalize the last layer of the DINO head.
        Not normalizing leads to better performance but can make the training unstable.
        In our experiments, we typically set this paramater to False with vit_small and True with vit_base.""")
    parser.add_argument('--momentum_teacher', default=0.996, type=float, help="""Base EMA
        parameter for teacher update. The value is increased to 1 during training with cosine schedule.
        We recommend setting a higher value with small batches: for example use 0.9995 with batch size of 256.""")
    parser.add_argument('--use_bn_in_head', default=False, type=utils.bool_flag,
        help="Whether to use batch normalizations in projection head (Default: False)")

    # Temperature teacher parameters
    parser.add_argument('--warmup_teacher_temp', default=0.04, type=float,
        help="""Initial value for the teacher temperature: 0.04 works well in most cases.
        Try decreasing it if the training loss does not decrease.""")
    parser.add_argument('--teacher_temp', default=0.04, type=float, help="""Final value (after linear warmup)
        of the teacher temperature. For most experiments, anything above 0.07 is unstable. We recommend
        starting with the default value of 0.04 and increase this slightly if needed.""")
    parser.add_argument('--warmup_teacher_temp_epochs', default=0, type=int,
        help='Number of warmup epochs for the teacher temperature (Default: 30).')

    # Training/Optimization parameters
    parser.add_argument('--use_fp16', type=utils.bool_flag, default=True, help="""Whether or not
        to use half precision for training. Improves training time and memory requirements,
        but can provoke instability and slight decay of performance. We recommend disabling
        mixed precision if the loss is unstable, if reducing the patch size or if training with bigger ViTs.""")
    parser.add_argument('--weight_decay', type=float, default=0.04, help="""Initial value of the
        weight decay. With ViT, a smaller value at the beginning of training works well.""")
    parser.add_argument('--weight_decay_end', type=float, default=0.4, help="""Final value of the
        weight decay. We use a cosine schedule for WD and using a larger decay by
        the end of training improves performance for ViTs.""")
    parser.add_argument('--clip_grad', type=float, default=3.0, help="""Maximal parameter
        gradient norm if using gradient clipping. Clipping with norm .3 ~ 1.0 can
        help optimization for larger ViT architectures. 0 for disabling.""")
    parser.add_argument('--batch_size_per_gpu', default=32, type=int,
        help='Per-GPU batch-size : number of distinct images loaded on one GPU.')
    parser.add_argument('--epochs', default=100, type=int, help='Number of epochs of training.')
    parser.add_argument('--freeze_last_layer', default=1, type=int, help="""Number of epochs
        during which we keep the output layer fixed. Typically doing so during
        the first epoch helps training. Try increasing this value if the loss does not decrease.""")
    parser.add_argument("--lr", default=0.0005, type=float, help="""Learning rate at the end of
        linear warmup (highest LR used during training). The learning rate is linearly scaled
        with the batch size, and specified here for a reference batch size of 256.""")
    parser.add_argument("--warmup_epochs", default=10, type=int,
        help="Number of epochs for the linear learning-rate warm up.")
    parser.add_argument('--min_lr', type=float, default=1e-6, help="""Target LR at the
        end of optimization. We use a cosine LR schedule with linear warmup.""")
    parser.add_argument('--optimizer', default='adamw', type=str,
        choices=['adamw', 'sgd', 'lars'], help="""Type of optimizer. We recommend using adamw with ViTs.""")
    parser.add_argument('--drop_path_rate', type=float, default=0.1, help="stochastic depth rate")

    # Multi-crop parameters
    parser.add_argument('--global_crops_scale', type=float, nargs='+', default=(0.4, 1.),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for large global view cropping. When disabling multi-crop (--local_crops_number 0), we
        recommand using a wider range of scale ("--global_crops_scale 0.14 1." for example)""")
    parser.add_argument('--local_crops_number', type=int, default=8, help="""Number of small
        local views to generate. Set this parameter to 0 to disable multi-crop training.
        When disabling multi-crop we recommend to use "--global_crops_scale 0.14 1." """)
    parser.add_argument('--local_crops_scale', type=float, nargs='+', default=(0.05, 0.4),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for small local view cropping of multi-crop.""")

    # Misc
    parser.add_argument('--data_path', default='/content/drive/MyDrive/datasets/CRC/train2/', type=str,
        help='Please specify path to the ImageNet training data.')
    parser.add_argument('--output_dir', default=".", type=str, help='Path to save logs and checkpoints.')
    parser.add_argument('--saveckp_freq', default=1, type=int, help='Save checkpoint every x epochs.')
    parser.add_argument('--seed', default=0, type=int, help='Random seed.')
    parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    # parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")

    # parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='dino_vitbase16_pretrain_full_checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    # parser.add_argument('--arch', default='vit_base', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    # parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    # parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set updistributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")




    return parser
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,)+(index,))
        return tuple_with_path


def train_dino(args):
    utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = DataAugmentationDINO(
        args.global_crops_scale,
        args.local_crops_scale,
        args.local_crops_number,
    )
    # dataset = datasets.ImageFolder(args.data_path, transform=transform)
    dataset = ImageFolderWithPaths(args.data_path, transform=transform)
    

    #################################################
    transformKNN = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    data_path="/content/drive/MyDrive/datasets/CRC/train2"
    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transformKNN)

    datasetKNN = ImageFolderWithPaths(args.data_path, transform=transformKNN)


    samplerKNN = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        # sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
        # ============ building network ... ============
    if "vit" in args.arch:
        knnModel = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        knnModel = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        knnModel = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        knnModel.cuda()
    knnModel.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = knnModel.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        knnModel.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################

    









    
    # class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    # dataset=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    sampler = torch.utils.data.DistributedSampler(dataset, shuffle=True)

    samplerkNN2 = torch.utils.data.DistributedSampler(datasetKNN, shuffle=True)

    
    data_loader = torch.utils.data.DataLoader(
        dataset,
        sampler=sampler,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    data_loaderKNN = torch.utils.data.DataLoader(
        datasetKNN,
        sampler=samplerkNN2,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    print(f"Data loaded: there are {len(dataset)} images.")
    # return data_loader
    # ============ building student and teacher networks ... ============
    # we changed the name DeiT-S for ViT-S to avoid confusions
    args.arch = args.arch.replace("deit", "vit")
    # if the network is a Vision Transformer (i.e. vit_tiny, vit_small, vit_base)
    if args.arch in vits.__dict__.keys():
        student = vits.__dict__[args.arch](
            patch_size=args.patch_size,
            drop_path_rate=args.drop_path_rate,  # stochastic depth
        )
        teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
        # student.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))
        # teacher.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))
        student.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))
        teacher.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))

        embed_dim = student.embed_dim
    # if the network is a XCiT
    elif args.arch in torch.hub.list("facebookresearch/xcit:main"):
        student = torch.hub.load('facebookresearch/xcit:main', args.arch,
                                 pretrained=False, drop_path_rate=args.drop_path_rate)
        teacher = torch.hub.load('facebookresearch/xcit:main', args.arch, pretrained=False)
        embed_dim = student.embed_dim
    # otherwise, we check if the architecture is in torchvision models
    elif args.arch in torchvision_models.__dict__.keys():
        student = torchvision_models.__dict__[args.arch]()
        teacher = torchvision_models.__dict__[args.arch]()
        embed_dim = student.fc.weight.shape[1]
        # embed_dim = student.state_dict()['encoder.layers.encoder_layer_11.mlp.3.weight'].shape[1]
        # embed_dim=1000
    else:
        print(f"Unknow architecture: {args.arch}")

    # multi-crop wrapper handles forward with inputs of different resolutions
    student = utils.MultiCropWrapper(student, DINOHead(
        embed_dim,
        args.out_dim,
        use_bn=args.use_bn_in_head,
        norm_last_layer=args.norm_last_layer,
    ))
    teacher = utils.MultiCropWrapper(
        teacher,
        DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
    )
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    # synchronize batch norms (if any)
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student = nn.parallel.DistributedDataParallel(student, device_ids=[args.gpu])
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.module.state_dict())
    # there is no backpropagation through the teacher, so no need for gradients
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

    # ============ preparing optimizer ... ============
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    # ============ init schedulers ... ============
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in range(start_epoch, args.epochs):
        data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)

        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")

        #################################
        torch.save(data_loader_train.dataset.img_list, 'img_namesAll.pt')
        train_features,train_label = extract_features(teacher, data_loaderKNN, args.use_cuda, multiscale=args.multiscale)
        # query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
        torch.save(train_features, 'train_featuresAll.pt')


        train_features = nn.functional.normalize(train_features, dim=1, p=2)

        sim = torch.mm(train_features, train_features.T)


        for i in range(len(data_loader_train.dataset.img_list)):
            # print(i)
            for j in range(len(data_loader_train.dataset.img_list)):
                index1=data_loader_train.dataset.img_list[i].rfind('/')
                index2=data_loader_train.dataset.img_list[j].rfind('/')
                if data_loader_train.dataset.img_list[i][index1+4:index1+7] == data_loader_train.dataset.img_list[j][index2+4:index2+7] and i!=j :
                    sim[i][j]=0

        KNN=[]
        for i in range(len(sim)):
          # print(i)
          KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:6])
        #################################

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))
    








def train_one_epoch(student, teacher, teacher_without_ddp, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,epoch,
                    fp16_scaler, args):
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}/{}]'.format(epoch, args.epochs)
    for it, (images, _,paths,indces) in enumerate(metric_logger.log_every(data_loader, 10, header)):
        # update weight decay and learning rate according to their schedule
        it = len(data_loader) * epoch + it  # global training iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]

        # move images to gpu
        images = [im.cuda(non_blocking=True) for im in images]
        # teacher and student forward passes + compute dino loss
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            
            teacher_output = teacher(images[:2])  # only the 2 global views pass through the teacher
            student_output = student(images[:])
            loss = dino_loss(student_output, teacher_output, epoch)

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)
        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            optimizer.step()
        else:
            fp16_scaler.scale(loss).backward()
            if args.clip_grad:
                fp16_scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            fp16_scaler.step(optimizer)
            fp16_scaler.update()

        # EMA update for the teacher
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            for param_q, param_k in zip(student.module.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)

        # logging
        torch.cuda.synchronize()
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
        dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output) * dist.get_world_size())

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)


class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(0.1),
            utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops


if __name__ == '__main__':
    parser = argparse.ArgumentParser('DINO', parents=[get_args_parser()])
    args = parser.parse_args("")
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    data_loader=train_dino(args)

Will run the code on one GPU.
| distributed init (rank 0): env://


Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


git:
  sha: N/A, status: clean, branch: N/A

arch: vit_base
batch_size_per_gpu: 32
checkpoint_key: teacher
clip_grad: 3.0
data_path: /content/drive/MyDrive/datasets/CRC/train2/
dataset: roxford5k
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
imsize: 224
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 1
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
multiscale: False
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 512
output_dir: .
patch_size: 16
pretrained_weights: dino_vitbase16_pretrain_full_checkpoint.pth
rank: 0
saveckp_freq: 1
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_cuda: True
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
['s1', 's2', 's3']


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model vit_base 16x16 built.
Data loaded: there are 41 images.
Student and Teacher are built: they are both vit_base network.
Loss, optimizer and schedulers ready.
Found checkpoint at ./checkpoint.pth
=> loaded 'student' from checkpoint './checkpoint.pth' with msg _IncompatibleKeys(missing_keys=['module.head.last_layer.weight_g', 'module.head.last_layer.weight_v'], unexpected_keys=['module.head.last_layer.weight'])
=> loaded 'teacher' from checkpoint './checkpoint.pth' with msg _IncompatibleKeys(missing_keys=['head.last_layer.weight_g', 'head.last_layer.weight_v'], unexpected_keys=['head.last_layer.weight'])
=> key 'optimizer' not found in checkpoint: './checkpoint.pth'
=> key 'fp16_scaler' not found in checkpoint: './checkpoint.pth'
=> key 'dino_loss' not found in checkpoint: './checkpoint.pth'
Starting DINO training !
Epoch: [0/100]  [0/1]  eta: 0:00:05  loss: 5.821088 (5.821088)  lr: 0.000000 (0.000000)  wd: 0.040000 (0.040000)  time: 5.876425  data: 1.773088  max mem: 11232
Epoch: [

RuntimeError: ignored

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path
from dataset import SingleData
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models

import utils
import vision_transformer as vits
from vision_transformer import DINOHead
from numpy import load

def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index,self.img_list[index]


KNN=load('/content/drive/MyDrive/models/dino-main/KNN2.npy')
torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))

def get_args_parser():
    parser = argparse.ArgumentParser('DINO', add_help=False)

    # Model parameters
    parser.add_argument('--arch', default='vit_base', type=str,
        choices=['vit_tiny', 'vit_small', 'vit_base', 'xcit', 'deit_tiny', 'deit_small'] \
                + torchvision_archs + torch.hub.list("facebookresearch/xcit:main"),
        help="""Name of architecture to train. For quick experiments with ViTs,
        we recommend using vit_tiny or vit_small.""")
    parser.add_argument('--patch_size', default=16, type=int, help="""Size in pixels
        of input square patches - default 16 (for 16x16 patches). Using smaller
        values leads to better performance but requires more memory. Applies only
        for ViTs (vit_tiny, vit_small and vit_base). If <16, we recommend disabling
        mixed precision training (--use_fp16 false) to avoid unstabilities.""")
    parser.add_argument('--out_dim', default=512, type=int, help="""Dimensionality of
        the DINO head output. For complex and large datasets large values (like 65k) work well.""")
    parser.add_argument('--norm_last_layer', default=True, type=utils.bool_flag,
        help="""Whether or not to weight normalize the last layer of the DINO head.
        Not normalizing leads to better performance but can make the training unstable.
        In our experiments, we typically set this paramater to False with vit_small and True with vit_base.""")
    parser.add_argument('--momentum_teacher', default=0.996, type=float, help="""Base EMA
        parameter for teacher update. The value is increased to 1 during training with cosine schedule.
        We recommend setting a higher value with small batches: for example use 0.9995 with batch size of 256.""")
    parser.add_argument('--use_bn_in_head', default=False, type=utils.bool_flag,
        help="Whether to use batch normalizations in projection head (Default: False)")

    # Temperature teacher parameters
    parser.add_argument('--warmup_teacher_temp', default=0.04, type=float,
        help="""Initial value for the teacher temperature: 0.04 works well in most cases.
        Try decreasing it if the training loss does not decrease.""")
    parser.add_argument('--teacher_temp', default=0.04, type=float, help="""Final value (after linear warmup)
        of the teacher temperature. For most experiments, anything above 0.07 is unstable. We recommend
        starting with the default value of 0.04 and increase this slightly if needed.""")
    parser.add_argument('--warmup_teacher_temp_epochs', default=0, type=int,
        help='Number of warmup epochs for the teacher temperature (Default: 30).')

    # Training/Optimization parameters
    parser.add_argument('--use_fp16', type=utils.bool_flag, default=True, help="""Whether or not
        to use half precision for training. Improves training time and memory requirements,
        but can provoke instability and slight decay of performance. We recommend disabling
        mixed precision if the loss is unstable, if reducing the patch size or if training with bigger ViTs.""")
    parser.add_argument('--weight_decay', type=float, default=0.04, help="""Initial value of the
        weight decay. With ViT, a smaller value at the beginning of training works well.""")
    parser.add_argument('--weight_decay_end', type=float, default=0.4, help="""Final value of the
        weight decay. We use a cosine schedule for WD and using a larger decay by
        the end of training improves performance for ViTs.""")
    parser.add_argument('--clip_grad', type=float, default=3.0, help="""Maximal parameter
        gradient norm if using gradient clipping. Clipping with norm .3 ~ 1.0 can
        help optimization for larger ViT architectures. 0 for disabling.""")
    parser.add_argument('--batch_size_per_gpu', default=32, type=int,
        help='Per-GPU batch-size : number of distinct images loaded on one GPU.')
    parser.add_argument('--epochs', default=100, type=int, help='Number of epochs of training.')
    parser.add_argument('--freeze_last_layer', default=1, type=int, help="""Number of epochs
        during which we keep the output layer fixed. Typically doing so during
        the first epoch helps training. Try increasing this value if the loss does not decrease.""")
    parser.add_argument("--lr", default=0.0005, type=float, help="""Learning rate at the end of
        linear warmup (highest LR used during training). The learning rate is linearly scaled
        with the batch size, and specified here for a reference batch size of 256.""")
    parser.add_argument("--warmup_epochs", default=10, type=int,
        help="Number of epochs for the linear learning-rate warm up.")
    parser.add_argument('--min_lr', type=float, default=1e-6, help="""Target LR at the
        end of optimization. We use a cosine LR schedule with linear warmup.""")
    parser.add_argument('--optimizer', default='adamw', type=str,
        choices=['adamw', 'sgd', 'lars'], help="""Type of optimizer. We recommend using adamw with ViTs.""")
    parser.add_argument('--drop_path_rate', type=float, default=0.1, help="stochastic depth rate")

    # Multi-crop parameters
    parser.add_argument('--global_crops_scale', type=float, nargs='+', default=(0.4, 1.),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for large global view cropping. When disabling multi-crop (--local_crops_number 0), we
        recommand using a wider range of scale ("--global_crops_scale 0.14 1." for example)""")
    parser.add_argument('--local_crops_number', type=int, default=8, help="""Number of small
        local views to generate. Set this parameter to 0 to disable multi-crop training.
        When disabling multi-crop we recommend to use "--global_crops_scale 0.14 1." """)
    parser.add_argument('--local_crops_scale', type=float, nargs='+', default=(0.05, 0.4),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for small local view cropping of multi-crop.""")

    # Misc
    parser.add_argument('--data_path', default='/content/drive/MyDrive/datasets/CRC/train2/', type=str,
        help='Please specify path to the ImageNet training data.')
    parser.add_argument('--output_dir', default=".", type=str, help='Path to save logs and checkpoints.')
    parser.add_argument('--saveckp_freq', default=1, type=int, help='Save checkpoint every x epochs.')
    parser.add_argument('--seed', default=0, type=int, help='Random seed.')
    parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    # parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")

    # parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    # parser.add_argument('--arch', default='vit_base', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    # parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    # parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set updistributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")




    return parser
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,)+(index,))
        return tuple_with_path


def train_dino(args):
    utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = DataAugmentationDINO(
        args.global_crops_scale,
        args.local_crops_scale,
        args.local_crops_number,
    )
    # dataset = datasets.ImageFolder(args.data_path, transform=transform)
    dataset = ImageFolderWithPaths(args.data_path, transform=transform)

    #################################################
    transformKNN = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    data_path="/content/drive/MyDrive/datasets/CRC/train2"
    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transformKNN)




    samplerKNN = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        sampler=samplerKNN,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
        # ============ building network ... ============
    if "vit" in args.arch:
        knnModel = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        knnModel = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        knnModel = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        knnModel.cuda()
    knnModel.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = knnModel.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        knnModel.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################

    









    
    # class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    # dataset=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    sampler = torch.utils.data.DistributedSampler(dataset, shuffle=True)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        sampler=sampler,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    print(f"Data loaded: there are {len(dataset)} images.")
    # return data_loader
    # ============ building student and teacher networks ... ============
    # we changed the name DeiT-S for ViT-S to avoid confusions
    args.arch = args.arch.replace("deit", "vit")
    # if the network is a Vision Transformer (i.e. vit_tiny, vit_small, vit_base)
    if args.arch in vits.__dict__.keys():
        student = vits.__dict__[args.arch](
            patch_size=args.patch_size,
            drop_path_rate=args.drop_path_rate,  # stochastic depth
        )
        teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
        # student.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))
        # teacher.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))
        student.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))
        teacher.load_state_dict(torch.load('/content/drive/MyDrive/models/dino-main/clipVitB16.pth'))

        embed_dim = student.embed_dim
    # if the network is a XCiT
    elif args.arch in torch.hub.list("facebookresearch/xcit:main"):
        student = torch.hub.load('facebookresearch/xcit:main', args.arch,
                                 pretrained=False, drop_path_rate=args.drop_path_rate)
        teacher = torch.hub.load('facebookresearch/xcit:main', args.arch, pretrained=False)
        embed_dim = student.embed_dim
    # otherwise, we check if the architecture is in torchvision models
    elif args.arch in torchvision_models.__dict__.keys():
        student = torchvision_models.__dict__[args.arch]()
        teacher = torchvision_models.__dict__[args.arch]()
        embed_dim = student.fc.weight.shape[1]
        # embed_dim = student.state_dict()['encoder.layers.encoder_layer_11.mlp.3.weight'].shape[1]
        # embed_dim=1000
    else:
        print(f"Unknow architecture: {args.arch}")

    # multi-crop wrapper handles forward with inputs of different resolutions
    student = utils.MultiCropWrapper(student, DINOHead(
        embed_dim,
        args.out_dim,
        use_bn=args.use_bn_in_head,
        norm_last_layer=args.norm_last_layer,
    ))
    teacher = utils.MultiCropWrapper(
        teacher,
        DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
    )
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    # synchronize batch norms (if any)
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student = nn.parallel.DistributedDataParallel(student, device_ids=[args.gpu])
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.module.state_dict())
    # there is no backpropagation through the teacher, so no need for gradients
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

    # ============ preparing optimizer ... ============
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    # ============ init schedulers ... ============
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in range(start_epoch, args.epochs):
        data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)

        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))
    torch.save(data_loader_train.dataset.img_list, 'img_namesAll.pt')
    train_features,train_label = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    # query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_featuresAll.pt')


    train_features = nn.functional.normalize(train_features, dim=1, p=2)

    sim = torch.mm(train_features, train_features.T)


    for i in range(len(data_loader_train.dataset.img_list)):
        print(i)
        for j in range(len(test_img_list)):
            index1=data_loader_train.dataset.img_list[i].rfind('/')
            index2=data_loader_train.dataset.img_list[j].rfind('/')
            if data_loader_train.dataset.img_list[i][index1+4:index1+7] == data_loader_train.dataset.img_list[j][index2+4:index2+7] and i!=j :
                sim[i][j]=0

    KNN=[]
    for i in range(len(sim)):
      print(i)
      KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:6])








def train_one_epoch(student, teacher, teacher_without_ddp, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,epoch,
                    fp16_scaler, args):
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}/{}]'.format(epoch, args.epochs)
    for it, (images, _,paths,indces) in enumerate(metric_logger.log_every(data_loader, 10, header)):
        # update weight decay and learning rate according to their schedule
        # print('DDD=>',dd)
        # print('images=>',paths)
        # print(index)
        # print(images[0])
        # for index in indces:
        #   temp=[]
        #   for 
        # np.random.randint(1,5)
        images2,b,c,d=data_loader.dataset.__getitem__(KNN[indces[0]][0])
        for i in range(len(images2)):
          images2[i]=images2[i].unsqueeze(0)

        nearst=[]
        for i in range(1,len(indces)):
          a,b,c,d=data_loader.dataset.__getitem__(KNN[indces[i]][0])
          nearst.append(a)
        
        for i in range(len(images2)):
          for j in range(len(nearst)):
            images2[i]=torch.cat((images2[i], nearst[j][i].unsqueeze(0)), 0)
            # print(images2[i].shape)
        # images2=[torch.Tensor() for _ in range(len(images))]
        # for i in range(len(images2)):
        #   for index in range(1,len(indces)):
        #     a,b,c,d=data_loader.dataset.__getitem__(index)
        #     images2[i]=torch.cat((images2[i], a[i].unsqueeze(0)), 0)
        #     print(images2[i].shape)

        # print(len(images2))
        # print(len(images2[0]))
        # print(images[0].shape)
        it = len(data_loader) * epoch + it  # global training iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]

        # move images to gpu
        images = [im.cuda(non_blocking=True) for im in images]
        images2 = [im.cuda(non_blocking=True) for im in images2]
        
        # teacher and student forward passes + compute dino loss
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            teacher_output = teacher(images[:2])  # only the 2 global views pass through the teacher
            student_output = student(images2)
            loss = dino_loss(student_output, teacher_output, epoch)

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)

        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            optimizer.step()
        else:
            fp16_scaler.scale(loss).backward()
            if args.clip_grad:
                fp16_scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            fp16_scaler.step(optimizer)
            fp16_scaler.update()

        # EMA update for the teacher
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            for param_q, param_k in zip(student.module.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)

        # logging
        torch.cuda.synchronize()
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
        dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output) * dist.get_world_size())

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)


class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(0.1),
            utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops


if __name__ == '__main__':
    parser = argparse.ArgumentParser('DINO', parents=[get_args_parser()])
    args = parser.parse_args("")
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    data_loader=train_dino(args)

Will run the code on one GPU.
| distributed init (rank 0): env://


Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


git:
  sha: N/A, status: clean, branch: N/A

arch: vit_base
batch_size_per_gpu: 32
checkpoint_key: teacher
clip_grad: 3.0
data_path: /content/drive/MyDrive/datasets/CRC/train2/
dataset: roxford5k
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
imsize: 224
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 1
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
multiscale: False
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 512
output_dir: .
patch_size: 16
pretrained_weights: /content/drive/MyDrive/models/dino-main/checkpoint.pth
rank: 0
saveckp_freq: 1
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_cuda: True
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
['s1', 's2', 's3']


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model vit_base 16x16 built.
Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/models/dino-main/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Data loaded: there are 34 images.
Student and Teacher are built: they are both vit_base network.
Loss, optimizer and schedulers ready.
Found checkpoint at ./checkpoint.pth
=> loaded 'student' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
=> loaded 'teacher' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
=> loaded 'optimizer' from checkpoint: './checkpoint.pth'
=> loaded 'fp16_scaler' from checkpoint: './checkpoint.pth'
=> loaded 'dino_loss' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
Starting DINO 

IndexError: ignored

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from google.colab import drive
drive.mount('/content/drive')
!pwd 
%cd drive/MyDrive/models/dino-main/
!nvidia-smi
!pip install timm==0.4.9
!pip install yacs
!pip install -U PyYAML
!pip install einops



# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np
from dataset import SingleData
import utils
from utils2 import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
from torchvision import datasets, transforms
import vision_transformer as vits
from eval_knn import extract_features
import math


def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

# class OxfordParisDataset(torch.utils.data.Dataset):
#     def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
#         if dataset not in ['roxford5k', 'rparis6k']:
#             raise ValueError('Unknown dataset: {}!'.format(dataset))

#         # loading imlist, qimlist, and gnd, in cfg as a dict
#         gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
#         with open(gnd_fname, 'rb') as f:
#             cfg = pickle.load(f)
#         cfg['gnd_fname'] = gnd_fname
#         cfg['ext'] = '.jpg'
#         cfg['qext'] = '.jpg'
#         cfg['dir_data'] = os.path.join(dir_main, dataset)
#         cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
#         cfg['n'] = len(cfg['imlist'])
#         cfg['nq'] = len(cfg['qimlist'])
#         cfg['im_fname'] = config_imname
#         cfg['qim_fname'] = config_qimname
#         cfg['dataset'] = dataset
#         self.cfg = cfg

#         self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
#         self.transform = transform
#         self.imsize = imsize

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, index):
#         path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
#         ImageFile.LOAD_TRUNCATED_IMAGES = True
#         with open(path, 'rb') as f:
#             img = Image.open(f)
#             img = img.convert('RGB')
#         if self.imsize is not None:
#             img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
#         if self.transform is not None:
#             img = self.transform(img)
#         return img, index


class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index,self.img_list[index]




def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_base', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    #################################################
    # data_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/train"
    # data_path="/content/drive/MyDrive/dataSets/CRC/forDINO/train"
    data_path="/content/drive/MyDrive/datasets/CRC/train"
   
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed different test2"
    test_path="/content/drive/MyDrive/datasets/CRC/train"

    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed same test"
    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    class_test_name, test_img_list = get_data_list(test_path,0.0)
    test_img_list=sorted(test_img_list)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transform)
    # train_sample = MPerClassSampler(train_data_set.label_list, batch_size=8)
    # data_loader_train = torch.utils.data.DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # train_data_set = datasets.ImageFolder(data_path, transform=transform)
    # test_data_set = datasets.ImageFolder(test_path, transform=transform)
    dataset_query = OxfordParisDataset(class_test_name, test_img_list, transform=transform)
    # data_loader_query = torch.utils.data.DataLoader(test_data_set, batch_size=8, shuffle=False, num_workers=8)
    ##################################################
    
    # dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    # dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    # sampler = torch.utils.data.DistributedSampler(train_data_set, shuffle=False)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        # sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        # test_data_set,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    # print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")
    # print(data_loader_train.dataset.samples)
    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    # Step 1: extract features
    print("###########",data_loader_query.dataset.img_list)
    torch.save(data_loader_query.dataset.img_list, 'img_namesAll.pt')
    train_features,train_label = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    # query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_featuresAll.pt')
    # torch.save(query_features, 'query_featuresAll.pt')
    torch.save(train_label, 'train_labelAll.pt')
    # torch.save(query_label, 'query_labelAll.pt')
    

    
    
    # print(train_features)
    # print(query_features)
    # if utils.get_rank() == 0:  # only rank 0 will work from now on
    #     # normalize features
    #     train_features = nn.functional.normalize(train_features, dim=1, p=2)
    #     query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        # sim = torch.mm(train_features, query_features.T)
        # ranks = torch.argsort(-sim, dim=0).cpu().numpy()
        # torch.save(sim, 'sim.pt')
        # print(sim)

        # print(ranks)


In [ ]:
l 

# Normal Train

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path
from dataset import SingleData
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
from eval_knn import extract_features
import utils
import vision_transformer as vits
from vision_transformer import DINOHead
from numpy import load

def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index,self.img_list[index]


KNN=load('/content/drive/MyDrive/models/dino-main/KNN2.npy')
torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))

def get_args_parser():
    parser = argparse.ArgumentParser('DINO', add_help=False)

    # Model parameters
    parser.add_argument('--arch', default='vit_small', type=str,
        choices=['vit_tiny', 'vit_small', 'vit_base', 'xcit', 'deit_tiny', 'deit_small'] \
                + torchvision_archs + torch.hub.list("facebookresearch/xcit:main"),
        help="""Name of architecture to train. For quick experiments with ViTs,
        we recommend using vit_tiny or vit_small.""")
    parser.add_argument('--patch_size', default=8, type=int, help="""Size in pixels
        of input square patches - default 16 (for 16x16 patches). Using smaller
        values leads to better performance but requires more memory. Applies only
        for ViTs (vit_tiny, vit_small and vit_base). If <16, we recommend disabling
        mixed precision training (--use_fp16 false) to avoid unstabilities.""")
    parser.add_argument('--out_dim', default=65536, type=int, help="""Dimensionality of
        the DINO head output. For complex and large datasets large values (like 65k) work well.""")
    parser.add_argument('--norm_last_layer', default=True, type=utils.bool_flag,
        help="""Whether or not to weight normalize the last layer of the DINO head.
        Not normalizing leads to better performance but can make the training unstable.
        In our experiments, we typically set this paramater to False with vit_small and True with vit_base.""")
    parser.add_argument('--momentum_teacher', default=0.996, type=float, help="""Base EMA
        parameter for teacher update. The value is increased to 1 during training with cosine schedule.
        We recommend setting a higher value with small batches: for example use 0.9995 with batch size of 256.""")
    parser.add_argument('--use_bn_in_head', default=False, type=utils.bool_flag,
        help="Whether to use batch normalizations in projection head (Default: False)")

    # Temperature teacher parameters
    parser.add_argument('--warmup_teacher_temp', default=0.04, type=float,
        help="""Initial value for the teacher temperature: 0.04 works well in most cases.
        Try decreasing it if the training loss does not decrease.""")
    parser.add_argument('--teacher_temp', default=0.04, type=float, help="""Final value (after linear warmup)
        of the teacher temperature. For most experiments, anything above 0.07 is unstable. We recommend
        starting with the default value of 0.04 and increase this slightly if needed.""")
    parser.add_argument('--warmup_teacher_temp_epochs', default=0, type=int,
        help='Number of warmup epochs for the teacher temperature (Default: 30).')

    # Training/Optimization parameters
    parser.add_argument('--use_fp16', type=utils.bool_flag, default=True, help="""Whether or not
        to use half precision for training. Improves training time and memory requirements,
        but can provoke instability and slight decay of performance. We recommend disabling
        mixed precision if the loss is unstable, if reducing the patch size or if training with bigger ViTs.""")
    parser.add_argument('--weight_decay', type=float, default=0.04, help="""Initial value of the
        weight decay. With ViT, a smaller value at the beginning of training works well.""")
    parser.add_argument('--weight_decay_end', type=float, default=0.4, help="""Final value of the
        weight decay. We use a cosine schedule for WD and using a larger decay by
        the end of training improves performance for ViTs.""")
    parser.add_argument('--clip_grad', type=float, default=3.0, help="""Maximal parameter
        gradient norm if using gradient clipping. Clipping with norm .3 ~ 1.0 can
        help optimization for larger ViT architectures. 0 for disabling.""")
    parser.add_argument('--batch_size_per_gpu', default=8, type=int,
        help='Per-GPU batch-size : number of distinct images loaded on one GPU.')
    parser.add_argument('--epochs', default=100, type=int, help='Number of epochs of training.')
    parser.add_argument('--freeze_last_layer', default=1, type=int, help="""Number of epochs
        during which we keep the output layer fixed. Typically doing so during
        the first epoch helps training. Try increasing this value if the loss does not decrease.""")
    parser.add_argument("--lr", default=0.0005, type=float, help="""Learning rate at the end of
        linear warmup (highest LR used during training). The learning rate is linearly scaled
        with the batch size, and specified here for a reference batch size of 256.""")
    parser.add_argument("--warmup_epochs", default=10, type=int,
        help="Number of epochs for the linear learning-rate warm up.")
    parser.add_argument('--min_lr', type=float, default=1e-6, help="""Target LR at the
        end of optimization. We use a cosine LR schedule with linear warmup.""")
    parser.add_argument('--optimizer', default='adamw', type=str,
        choices=['adamw', 'sgd', 'lars'], help="""Type of optimizer. We recommend using adamw with ViTs.""")
    parser.add_argument('--drop_path_rate', type=float, default=0.1, help="stochastic depth rate")

    # Multi-crop parameters
    parser.add_argument('--global_crops_scale', type=float, nargs='+', default=(0.4, 1.),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for large global view cropping. When disabling multi-crop (--local_crops_number 0), we
        recommand using a wider range of scale ("--global_crops_scale 0.14 1." for example)""")
    parser.add_argument('--local_crops_number', type=int, default=8, help="""Number of small
        local views to generate. Set this parameter to 0 to disable multi-crop training.
        When disabling multi-crop we recommend to use "--global_crops_scale 0.14 1." """)
    parser.add_argument('--local_crops_scale', type=float, nargs='+', default=(0.05, 0.4),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for small local view cropping of multi-crop.""")

    # Misc
    parser.add_argument('--data_path', default='/content/drive/MyDrive/datasets/CRC/train/', type=str,
        help='Please specify path to the ImageNet training data.')
    parser.add_argument('--output_dir', default=".", type=str, help='Path to save logs and checkpoints.')
    parser.add_argument('--saveckp_freq', default=2, type=int, help='Save checkpoint every x epochs.')
    parser.add_argument('--seed', default=0, type=int, help='Random seed.')
    parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    # parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")

    # parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    # parser.add_argument('--arch', default='vit_base', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    # parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    # parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set updistributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")




    return parser
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,)+(index,))
        return tuple_with_path


def train_dino(args):
    utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = DataAugmentationDINO(
        args.global_crops_scale,
        args.local_crops_scale,
        args.local_crops_number,
    )
    # dataset = datasets.ImageFolder(args.data_path, transform=transform)
    dataset = ImageFolderWithPaths(args.data_path, transform=transform)
    

    #################################################
    transformKNN = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    data_path="/content/drive/MyDrive/datasets/CRC/test"
    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    # dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transformKNN)

    datasetKNN = ImageFolderWithPaths(data_path, transform=transformKNN)


    # samplerKNN = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    # data_loader_train = torch.utils.data.DataLoader(
    #     dataset_train,
    #     # train_data_set,
    #     # sampler=sampler,
    #     # sampler=train_sample,
    #     batch_size=1,
    #     num_workers=args.num_workers,
    #     pin_memory=True,
    #     drop_last=False,
    # )
        # ============ building network ... ============
    if "vit" in args.arch:
        knnModel = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        knnModel = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        knnModel = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        knnModel.cuda()
    knnModel.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = knnModel.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        knnModel.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################

    









    
    # class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    # dataset=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    sampler = torch.utils.data.DistributedSampler(dataset, shuffle=True)

    samplerkNN2 = torch.utils.data.DistributedSampler(datasetKNN, shuffle=True)

    
    data_loader = torch.utils.data.DataLoader(
        dataset,
        sampler=sampler,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    data_loaderKNN = torch.utils.data.DataLoader(
        datasetKNN,
        sampler=samplerkNN2,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
        shuffle=False
        
    )
    # print("&&&&&")
    # print(data_loaderKNN.dataset.imgs[0])
    print(f"Data loaded: there are {len(dataset)} images.")
    # return data_loader
    # ============ building student and teacher networks ... ============
    # we changed the name DeiT-S for ViT-S to avoid confusions
    # we changed the name DeiT-S for ViT-S to avoid confusions
    args.arch = args.arch.replace("deit", "vit")
    # if the network is a Vision Transformer (i.e. vit_tiny, vit_small, vit_base)
    if args.arch in vits.__dict__.keys():
        student = vits.__dict__[args.arch](
            patch_size=args.patch_size,
            drop_path_rate=args.drop_path_rate,  # stochastic depth
        )
        teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
        embed_dim = student.embed_dim
    # if the network is a XCiT
    elif args.arch in torch.hub.list("facebookresearch/xcit:main"):
        student = torch.hub.load('facebookresearch/xcit:main', args.arch,
                                 pretrained=False, drop_path_rate=args.drop_path_rate)
        teacher = torch.hub.load('facebookresearch/xcit:main', args.arch, pretrained=False)
        embed_dim = student.embed_dim
    # otherwise, we check if the architecture is in torchvision models
    elif args.arch in torchvision_models.__dict__.keys():
        student = torchvision_models.__dict__[args.arch]()
        teacher = torchvision_models.__dict__[args.arch]()
        embed_dim = student.fc.weight.shape[1]
    else:
        print(f"Unknow architecture: {args.arch}")

    # multi-crop wrapper handles forward with inputs of different resolutions
    student = utils.MultiCropWrapper(student, DINOHead(
        embed_dim,
        args.out_dim,
        use_bn=args.use_bn_in_head,
        norm_last_layer=args.norm_last_layer,
    ))
    teacher = utils.MultiCropWrapper(
        teacher,
        DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
    )
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    # synchronize batch norms (if any)
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student = nn.parallel.DistributedDataParallel(student, device_ids=[args.gpu])
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.module.state_dict())
    # there is no backpropagation through the teacher, so no need for gradients
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

    # ============ preparing optimizer ... ============
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    # ============ init schedulers ... ============
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in range(start_epoch, args.epochs):
        data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)

        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")

        ################################
        # torch.save(data_loaderKNN.dataset.img_list, 'img_namesAll.pt')
        train_features,train_label = extract_features(teacher, data_loaderKNN, args.use_cuda, multiscale=args.multiscale)
        # query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
        torch.save(train_features, 'train_featuresAll.pt')


        train_features = nn.functional.normalize(train_features, dim=1, p=2)

        sim = torch.mm(train_features, train_features.T)


        for i in range(len(data_loaderKNN.dataset.imgs)):
            # print(i)
            for j in range(len(data_loaderKNN.dataset.imgs)):
                index1=data_loaderKNN.dataset.imgs[i][0].rfind('/')
                index2=data_loaderKNN.dataset.imgs[j][0].rfind('/')
                if data_loaderKNN.dataset.imgs[i][0][index1+4:index1+7] == data_loaderKNN.dataset.imgs[j][0][index2+4:index2+7] and i!=j :
                    sim[i][j]=0

        # KNN=[]
        # for i in range(len(sim)):
        #   # print(i)
        #   KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:6])
        simNumpy=sim.cpu().numpy()
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()

        # for i in range(300):
        #     sim[i][i]=-100.0

        # ranks2 = torch.argsort(-sim, dim=0).cpu().numpy()


        count = 0

        for i in range(1000):
            if ranks[1,i]<1000:
                count +=1
        print(count)        
        for i in range(1000,2000):
            if ranks[1,i]>1000 and ranks[1,i]<2000 :
                count +=1
                
        print(count)           
        for i in range(2000,3000):
            if ranks[1,i]>2000 and ranks[1,i]<3000 :
                count +=1
        print(count)  

        precent=count/3000
        print(precent)

        #################################

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))
    








def train_one_epoch(student, teacher, teacher_without_ddp, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,epoch,
                    fp16_scaler, args):
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}/{}]'.format(epoch, args.epochs)
    for it, (images, _,paths,indces) in enumerate(metric_logger.log_every(data_loader, 10, header)):
        # update weight decay and learning rate according to their schedule
        it = len(data_loader) * epoch + it  # global training iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]

        # move images to gpu
        images = [im.cuda(non_blocking=True) for im in images]
        # teacher and student forward passes + compute dino loss
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            
            teacher_output = teacher(images[:2])  # only the 2 global views pass through the teacher
            student_output = student(images)
            loss = dino_loss(student_output, teacher_output, epoch)

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)
        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            optimizer.step()
        else:
            fp16_scaler.scale(loss).backward()
            if args.clip_grad:
                fp16_scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            fp16_scaler.step(optimizer)
            fp16_scaler.update()

        # EMA update for the teacher
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            for param_q, param_k in zip(student.module.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)

        # logging
        torch.cuda.synchronize()
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
        dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output) * dist.get_world_size())

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)


class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(0.1),
            utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops


if __name__ == '__main__':
    parser = argparse.ArgumentParser('DINO', parents=[get_args_parser()])
    args = parser.parse_args("")
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    data_loader=train_dino(args)

Downloading: "https://github.com/facebookresearch/xcit/zipball/main" to /root/.cache/torch/hub/main.zip


Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: N/A, status: clean, branch: N/A

arch: vit_small
batch_size_per_gpu: 8
checkpoint_key: teacher
clip_grad: 3.0
data_path: /content/drive/MyDrive/datasets/CRC/train/
dataset: roxford5k
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
imsize: 224
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 1
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
multiscale: False
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: .
patch_size: 8
pretrained_weights: checkpoint.pth
rank: 0
saveckp_freq: 2
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_cuda: True
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


Model vit_small 8x8 built.
Take key teacher in provided checkpoint dict
Pretrained weights found at checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Data loaded: there are 42885 images.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Streaming output truncated to the last 5000 lines.
Epoch: [17/100]  [2620/5360]  eta: 0:43:28  loss: 1.794423 (1.859590)  lr: 0.000015 (0.000015)  wd: 0.066486 (0.065776)  time: 0.950616  data: 0.000248  max mem: 10380
Epoch: [17/100]  [2630/5360]  eta: 0:43:18  loss: 1.790720 (1.859017)  lr: 0.000015 (0.000015)  wd: 0.066492 (0.065779)  time: 0.950851  data: 0.000188  max mem: 10380
Epoch: [17/100]  [2640/5360]  eta: 0:43:09  loss: 1.661191 (1.858387)  lr: 0.000015 (0.000015)  wd: 0.066497 (0.065782)  time: 0.951167  data: 0.000203  max mem: 10380
Epoch: [17/100]  [2650/5360]  eta: 0:42:59  loss: 1.701373 (1.857921)  lr: 0.000015 (0.000015)  wd: 0.066503 (0.065784)  time: 0.951400  data: 0.000257  max mem: 10380
Epoch: [17/100]  [2660/5360]  eta: 0:42:50  loss: 1.728313 (1.858166)  lr: 0.000015 (0.000015)  wd: 0.066508 (0.065787)  time: 0.951223  data: 0.000212  max mem: 10380
Epoch: [17/100]  [2670/5360]  eta: 0:42:40  loss: 1.794259 (1.857743)  lr: 0.000015 (0.000015)  wd: 0.066514 

KeyboardInterrupt: ignored

# KNN

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from google.colab import drive
drive.mount('/content/drive')
!pwd 
%cd drive/MyDrive/models/dino-main/
!nvidia-smi
!pip install timm==0.4.9
!pip install yacs
!pip install -U PyYAML
!pip install einops



# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np
from dataset import SingleData
import utils
from utils2 import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
from torchvision import datasets, transforms
import vision_transformer as vits
from eval_knn import extract_features
import math


def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

# class OxfordParisDataset(torch.utils.data.Dataset):
#     def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
#         if dataset not in ['roxford5k', 'rparis6k']:
#             raise ValueError('Unknown dataset: {}!'.format(dataset))

#         # loading imlist, qimlist, and gnd, in cfg as a dict
#         gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
#         with open(gnd_fname, 'rb') as f:
#             cfg = pickle.load(f)
#         cfg['gnd_fname'] = gnd_fname
#         cfg['ext'] = '.jpg'
#         cfg['qext'] = '.jpg'
#         cfg['dir_data'] = os.path.join(dir_main, dataset)
#         cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
#         cfg['n'] = len(cfg['imlist'])
#         cfg['nq'] = len(cfg['qimlist'])
#         cfg['im_fname'] = config_imname
#         cfg['qim_fname'] = config_qimname
#         cfg['dataset'] = dataset
#         self.cfg = cfg

#         self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
#         self.transform = transform
#         self.imsize = imsize

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, index):
#         path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
#         ImageFile.LOAD_TRUNCATED_IMAGES = True
#         with open(path, 'rb') as f:
#             img = Image.open(f)
#             img = img.convert('RGB')
#         if self.imsize is not None:
#             img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
#         if self.transform is not None:
#             img = self.transform(img)
#         return img, index


class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index,self.img_list[index]

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,)+(index,))
        return tuple_with_path



def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_small', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument('--patch_size', default=8, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    transformKNN = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    #################################################
    # data_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/train"
    # data_path="/content/drive/MyDrive/dataSets/CRC/forDINO/train"
    data_path="/content/drive/MyDrive/datasets/CRC/train"
   
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed different test2"
    test_path="/content/drive/MyDrive/datasets/CRC/train"

    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed same test"
    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    class_test_name, test_img_list = get_data_list(test_path,0.0)
    test_img_list=sorted(test_img_list)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transform)
    # train_sample = MPerClassSampler(train_data_set.label_list, batch_size=8)
    # data_loader_train = torch.utils.data.DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # train_data_set = datasets.ImageFolder(data_path, transform=transform)
    # test_data_set = datasets.ImageFolder(test_path, transform=transform)
    dataset_query = OxfordParisDataset(class_test_name, test_img_list, transform=transform)
    # data_loader_query = torch.utils.data.DataLoader(test_data_set, batch_size=8, shuffle=False, num_workers=8)
    ##################################################
    
    # dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    # dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    # sampler = torch.utils.data.DistributedSampler(train_data_set, shuffle=False)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        # sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        # test_data_set,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    # print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")
    # print(data_loader_train.dataset.samples)
    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    data_path="/content/drive/MyDrive/datasets/CRC/test"
    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    datasetKNN = ImageFolderWithPaths(data_path, transform=transformKNN)

    samplerkNN2 = torch.utils.data.DistributedSampler(datasetKNN, shuffle=True)
    data_loaderKNN = torch.utils.data.DataLoader(
        datasetKNN,
        sampler=samplerkNN2,
        batch_size=64,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
        shuffle=False
        
    )
    # dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transformKNN)


    # Step 1: extract features
    print("###########",data_loader_query.dataset.img_list)
    torch.save(data_loaderKNN.dataset.imgs, 'img_namesAll.pt')
    train_features,train_label = extract_features(model, data_loaderKNN, args.use_cuda, multiscale=args.multiscale)
    # query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_featuresAll.pt')
    # torch.save(query_features, 'query_featuresAll.pt')
    torch.save(train_label, 'train_labelAll.pt')
    # torch.save(query_label, 'query_labelAll.pt')
    

    
    
    # print(train_features)
    # print(query_features)
    # if utils.get_rank() == 0:  # only rank 0 will work from now on
    #     # normalize features
    #     train_features = nn.functional.normalize(train_features, dim=1, p=2)
    #     query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        # sim = torch.mm(train_features, query_features.T)
        # ranks = torch.argsort(-sim, dim=0).cpu().numpy()
        # torch.save(sim, 'sim.pt')
        # print(sim)

        # print(ranks)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import torch
import torch.nn as nn
query_features=torch.load('/content/drive/MyDrive/models/dino-main/train_featuresAll.pt',map_location=torch.device('cpu'))
test_img_list=torch.load('/content/drive/MyDrive/models/dino-main/img_namesAll.pt',map_location=torch.device('cpu'))
# query_features = nn.functional.normalize(query_features, dim=1, p=2)


In [ ]:
import torch
import torch.nn as nn
query_features=torch.load('/content/drive/MyDrive/models/dino-main/train_featuresAll.pt',map_location=torch.device('cpu'))
test_img_list=torch.load('/content/drive/MyDrive/models/dino-main/img_namesAll.pt',map_location=torch.device('cpu'))
query_features = nn.functional.normalize(query_features, dim=1, p=2)

sim = torch.mm(query_features, query_features.T)


for i in range(len(test_img_list)):
        for j in range(len(test_img_list)):
            index1=test_img_list[i][0].rfind('/')
            index2=test_img_list[j][0].rfind('/')
            if test_img_list[i][0][index1+4:index1+7] == test_img_list[j][0][index2+4:index2+7] and i!=j:
                sim[i][j]=0

In [ ]:
del query_features
del test_img_list
 

In [ ]:
import torch
import torch.nn as nn
query_features=torch.load('/content/drive/MyDrive/models/dino-main/train_featuresAll.pt',map_location=torch.device('cpu'))
test_img_list=torch.load('/content/drive/MyDrive/models/dino-main/img_namesAll.pt',map_location=torch.device('cpu'))
query_features = nn.functional.normalize(query_features, dim=1, p=2)
query_features=query_features.cpu()
sim = torch.mm(query_features, query_features.T)


for i in range(len(test_img_list)):
        for j in range(len(test_img_list)):
            index1=test_img_list[i][0].rfind('/')
            index2=test_img_list[j][0].rfind('/')
            if test_img_list[i][0][index1+4:index1+7] == test_img_list[j][0][index2+4:index2+7] and i!=j:
                sim[i][j]=0


# for i in range(300):
#     sim[i][i]=-100.0





simNumpy=sim.cpu().numpy()
ranks = torch.argsort(-sim, dim=0).cpu().numpy()

# for i in range(300):
#     sim[i][i]=-100.0

# ranks2 = torch.argsort(-sim, dim=0).cpu().numpy()


count = 0

for i in range(1000):
    if ranks[1,i]<1000:
        count +=1
print(count)        
for i in range(1000,2000):
    if ranks[1,i]>1000 and ranks[1,i]<2000 :
        count +=1
        
print(count)           
for i in range(2000,3000):
    if ranks[1,i]>2000 and ranks[1,i]<3000 :
        count +=1
print(count)  

precent=count/3000
print(precent)


# count2 = 0

# for i in range(1000):
#     if ranks[2,i]<1000:
#         count2 +=1
# print(count2)        
# for i in range(1000,2000):
#     if ranks[2,i]>1000 and ranks[2,i]<2000 :
#         count2 +=1
        
# print(count2)           
# for i in range(2000,3000):
#     if ranks[2,i]>2000 and ranks[2,i]<3000 :
#         count2 +=1
# print(count2)           
                 
        
# count22 = 0

# for i in range(100):
#     if ranks2[0,i]<100:
#         count22 +=1
        
# for i in range(100,200):
#     if ranks2[0,i]>100 and ranks2[0,i]<200 :
#         count22 +=1
        
        
# for i in range(200,300):
#     if ranks2[0,i]>200 and ranks2[0,i]<300 :
#         count22 +=1
##################################use torch.cdist###########################        
        
# sim3=torch.cdist(query_features,query_features)


# for i in range(len(test_img_list)):
#         for j in range(len(test_img_list)):
#             index1=test_img_list[i].rfind('/')
#             index2=test_img_list[j].rfind('/')
#             if test_img_list[i][index1+4:index1+7] == test_img_list[j][index2+4:index2+7] and i!=j and test_img_list[i][index1+1:index1+3] == test_img_list[j][index2+1:index2+3] :
#                 sim3[i][j]=10000



# ranks3 = torch.argsort(sim3, dim=0).cpu().numpy()



# count3 = 0

# for i in range(100):
#     if ranks3[1,i]<100:
#         count3 +=1
        
# for i in range(100,200):
#     if ranks3[1,i]>100 and ranks3[1,i]<200 :
#         count3 +=1
        
        
# for i in range(200,300):
#     if ranks3[1,i]>200 and ranks3[1,i]<300 :
#         count3 +=1

# import numpy as np

# z=ranks[1,:]
# a=np.zeros((3000,))
# for i in range(len(z)):
#     if(z[i]<1000):
#         z[i]=0
#     if(z[i]>=1000 and z[i]<2000):
#         z[i]=1
#     if(z[i]>=2000):
#         z[i]=2


# for i in range(len(a)):
#     if(i<1000):
#         a[i]=0
#     if(i>=1000 and i<2000):
#         a[i]=1
#     if(i>=2000):
#         a[i]=2        
        
# cnf_mat = np.zeros((3, 3))       
        
# for i in range(len(a)):
#         cnf_mat[int(a[i])][int(z[i])] += 1
        
# e = cnf_mat.diagonal()
# n_w = (e/cnf_mat.sum(axis=1)).mean()

775
1394
1957
0.6523333333333333


In [ ]:
from numpy import load
mask=load('/content/drive/MyDrive/models/dino-main/mask.npy')
len(mask)

34419432

In [ ]:
count=0
for i in range(10):
  for j in range(10):
    if i+j<10:
      print("1", end = '')
      count+=1
    else:
      print("0", end = '')
  print("")

print(count)

1111111111
1111111110
1111111100
1111111000
1111110000
1111100000
1111000000
1110000000
1100000000
1000000000
55


In [ ]:
import torch
import torch.nn as nn
from numpy import load

train_features=torch.load('/content/drive/MyDrive/models/dino-main/train_featuresAll.pt',map_location=torch.device('cpu'))
test_img_list=torch.load('/content/drive/MyDrive/models/dino-main/img_namesAll.pt',map_location=torch.device('cpu'))
# train_features = nn.functional.normalize(train_features, dim=1, p=2)

# # query_features
sim = torch.mm(train_features, train_features.T)
mask=load('/content/drive/MyDrive/models/dino-main/mask.npy')
for i in mask:
  sim[i[0]][i[1]]=0
  # sim[i[1]][i[0]]=0

In [ ]:
import torch
import torch.nn as nn
# query_features=torch.load('/content/drive/MyDrive/models/dino-main/train_featuresAll.pt',map_location=torch.device('cpu'))
test_img_list=torch.load('/content/drive/MyDrive/models/dino-main/img_namesAll.pt',map_location=torch.device('cpu'))
# train_features = nn.functional.normalize(train_features, dim=1, p=2)

# # query_features
# sim = torch.mm(train_features, train_features.T)
# mask=load('/content/drive/MyDrive/models/dino-main/mask.npy')
sim = torch.mm(train_features, train_features.T)
mask=load('/content/drive/MyDrive/models/dino-main/effMask.npy')
for i in mask:
  # print(i)# for i in mask:

mask=[]


for i in range(len(test_img_list)):
        print(i)
        for j in range(i):
            index1=test_img_list[i].rfind('/')
            index2=test_img_list[j].rfind('/')
            if test_img_list[i][index1+4:index1+7] == test_img_list[j][index2+4:index2+7] and i!=j :
                mask.append((i,j))

Streaming output truncated to the last 5000 lines.
37885
37886
37887
37888
37889
37890
37891
37892
37893
37894
37895
37896
37897
37898
37899
37900
37901
37902
37903
37904
37905
37906
37907
37908
37909
37910
37911
37912
37913
37914
37915
37916
37917
37918
37919
37920
37921
37922
37923
37924
37925
37926
37927
37928
37929
37930
37931
37932
37933
37934
37935
37936
37937
37938
37939
37940
37941
37942
37943
37944
37945
37946
37947
37948
37949
37950
37951
37952
37953
37954
37955
37956
37957
37958
37959
37960
37961
37962
37963
37964
37965
37966
37967
37968
37969
37970
37971
37972
37973
37974
37975
37976
37977
37978
37979
37980
37981
37982
37983
37984
37985
37986
37987
37988
37989
37990
37991
37992
37993
37994
37995
37996
37997
37998
37999
38000
38001
38002
38003
38004
38005
38006
38007
38008
38009
38010
38011
38012
38013
38014
38015
38016
38017
38018
38019
38020
38021
38022
38023
38024
38025
38026
38027
38028
38029
38030
38031
38032
38033
38034
38035
38036
38037
38038
38039
38040
38041
38042
3

In [ ]:
from numpy import save
save('/content/drive/MyDrive/models/dino-main/effMask.npy', mask)

NameError: ignored

In [ ]:
import torch
import torch.nn as nn
train_features=torch.load('/content/drive/MyDrive/models/dino-main/train_featuresAll.pt',map_location=torch.device('cpu'))
test_img_list=torch.load('/content/drive/MyDrive/models/dino-main/img_namesAll.pt',map_location=torch.device('cpu'))
train_features = nn.functional.normalize(train_features, dim=1, p=2)

# query_features
from numpy import load
sim = torch.mm(train_features, train_features.T)
mask=load('/content/drive/MyDrive/models/dino-main/effMask.npy')
for i in mask:
  # print(i)
  sim[i[0]][i[1]]=0
  sim[i[1]][i[0]]=0


# for i in range(len(test_img_list)):
#         print(i)
#         for j in range(len(test_img_list)):
#             index1=test_img_list[i].rfind('/')
#             index2=test_img_list[j].rfind('/')
#             if test_img_list[i][index1+4:index1+7] == test_img_list[j][index2+4:index2+7] and i!=j :
#                 sim[i][j]=0

In [ ]:
del mask

In [ ]:
torch.argsort(-sim[:,:], dim=0)[:6].cpu().numpy()[:6])


In [ ]:
KNN=[]
for i in range(len(sim)):
  print(i)
  KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:3])

from numpy import save
save('/content/drive/MyDrive/models/dino-main/KNNeff.npy', KNN)

Streaming output truncated to the last 5000 lines.
37885
37886
37887
37888
37889
37890
37891
37892
37893
37894
37895
37896
37897
37898
37899
37900
37901
37902
37903
37904
37905
37906
37907
37908
37909
37910
37911
37912
37913
37914
37915
37916
37917
37918
37919
37920
37921
37922
37923
37924
37925
37926
37927
37928
37929
37930
37931
37932
37933
37934
37935
37936
37937
37938
37939
37940
37941
37942
37943
37944
37945
37946
37947
37948
37949
37950
37951
37952
37953
37954
37955
37956
37957
37958
37959
37960
37961
37962
37963
37964
37965
37966
37967
37968
37969
37970
37971
37972
37973
37974
37975
37976
37977
37978
37979
37980
37981
37982
37983
37984
37985
37986
37987
37988
37989
37990
37991
37992
37993
37994
37995
37996
37997
37998
37999
38000
38001
38002
38003
38004
38005
38006
38007
38008
38009
38010
38011
38012
38013
38014
38015
38016
38017
38018
38019
38020
38021
38022
38023
38024
38025
38026
38027
38028
38029
38030
38031
38032
38033
38034
38035
38036
38037
38038
38039
38040
38041
38042
3

In [ ]:
from numpy import save
save('/content/drive/MyDrive/models/dino-main/KNNeff.npy', KNN)

# train with KNN

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path
from dataset import SingleData
import numpy as np
from PIL import Image
from numpy import save
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
from eval_knn import extract_features
import utils
import vision_transformer as vits
from vision_transformer import DINOHead
from numpy import load
mask=load('/content/drive/MyDrive/models/dino-main/effMask.npy')

def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index,self.img_list[index]


KNN=load('/content/drive/MyDrive/models/dino-main/KNNeff.npy')

torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))

def get_args_parser():
    parser = argparse.ArgumentParser('DINO', add_help=False)

    # Model parameters
    parser.add_argument('--arch', default='vit_small', type=str,
        choices=['vit_tiny', 'vit_small', 'vit_base', 'xcit', 'deit_tiny', 'deit_small'] \
                + torchvision_archs + torch.hub.list("facebookresearch/xcit:main"),
        help="""Name of architecture to train. For quick experiments with ViTs,
        we recommend using vit_tiny or vit_small.""")
    parser.add_argument('--patch_size', default=8, type=int, help="""Size in pixels
        of input square patches - default 16 (for 16x16 patches). Using smaller
        values leads to better performance but requires more memory. Applies only
        for ViTs (vit_tiny, vit_small and vit_base). If <16, we recommend disabling
        mixed precision training (--use_fp16 false) to avoid unstabilities.""")
    parser.add_argument('--out_dim', default=65536, type=int, help="""Dimensionality of
        the DINO head output. For complex and large datasets large values (like 65k) work well.""")
    parser.add_argument('--norm_last_layer', default=True, type=utils.bool_flag,
        help="""Whether or not to weight normalize the last layer of the DINO head.
        Not normalizing leads to better performance but can make the training unstable.
        In our experiments, we typically set this paramater to False with vit_small and True with vit_base.""")
    parser.add_argument('--momentum_teacher', default=0.996, type=float, help="""Base EMA
        parameter for teacher update. The value is increased to 1 during training with cosine schedule.
        We recommend setting a higher value with small batches: for example use 0.9995 with batch size of 256.""")
    parser.add_argument('--use_bn_in_head', default=False, type=utils.bool_flag,
        help="Whether to use batch normalizations in projection head (Default: False)")

    # Temperature teacher parameters
    parser.add_argument('--warmup_teacher_temp', default=0.04, type=float,
        help="""Initial value for the teacher temperature: 0.04 works well in most cases.
        Try decreasing it if the training loss does not decrease.""")
    parser.add_argument('--teacher_temp', default=0.04, type=float, help="""Final value (after linear warmup)
        of the teacher temperature. For most experiments, anything above 0.07 is unstable. We recommend
        starting with the default value of 0.04 and increase this slightly if needed.""")
    parser.add_argument('--warmup_teacher_temp_epochs', default=0, type=int,
        help='Number of warmup epochs for the teacher temperature (Default: 30).')

    # Training/Optimization parameters
    parser.add_argument('--use_fp16', type=utils.bool_flag, default=True, help="""Whether or not
        to use half precision for training. Improves training time and memory requirements,
        but can provoke instability and slight decay of performance. We recommend disabling
        mixed precision if the loss is unstable, if reducing the patch size or if training with bigger ViTs.""")
    parser.add_argument('--weight_decay', type=float, default=0.04, help="""Initial value of the
        weight decay. With ViT, a smaller value at the beginning of training works well.""")
    parser.add_argument('--weight_decay_end', type=float, default=0.4, help="""Final value of the
        weight decay. We use a cosine schedule for WD and using a larger decay by
        the end of training improves performance for ViTs.""")
    parser.add_argument('--clip_grad', type=float, default=3.0, help="""Maximal parameter
        gradient norm if using gradient clipping. Clipping with norm .3 ~ 1.0 can
        help optimization for larger ViT architectures. 0 for disabling.""")
    parser.add_argument('--batch_size_per_gpu', default=8, type=int,
        help='Per-GPU batch-size : number of distinct images loaded on one GPU.')
    parser.add_argument('--epochs', default=100, type=int, help='Number of epochs of training.')
    parser.add_argument('--freeze_last_layer', default=1, type=int, help="""Number of epochs
        during which we keep the output layer fixed. Typically doing so during
        the first epoch helps training. Try increasing this value if the loss does not decrease.""")
    parser.add_argument("--lr", default=0.0005, type=float, help="""Learning rate at the end of
        linear warmup (highest LR used during training). The learning rate is linearly scaled
        with the batch size, and specified here for a reference batch size of 256.""")
    parser.add_argument("--warmup_epochs", default=10, type=int,
        help="Number of epochs for the linear learning-rate warm up.")
    parser.add_argument('--min_lr', type=float, default=1e-6, help="""Target LR at the
        end of optimization. We use a cosine LR schedule with linear warmup.""")
    parser.add_argument('--optimizer', default='adamw', type=str,
        choices=['adamw', 'sgd', 'lars'], help="""Type of optimizer. We recommend using adamw with ViTs.""")
    parser.add_argument('--drop_path_rate', type=float, default=0.1, help="stochastic depth rate")

    # Multi-crop parameters
    parser.add_argument('--global_crops_scale', type=float, nargs='+', default=(0.4, 1.),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for large global view cropping. When disabling multi-crop (--local_crops_number 0), we
        recommand using a wider range of scale ("--global_crops_scale 0.14 1." for example)""")
    parser.add_argument('--local_crops_number', type=int, default=8, help="""Number of small
        local views to generate. Set this parameter to 0 to disable multi-crop training.
        When disabling multi-crop we recommend to use "--global_crops_scale 0.14 1." """)
    parser.add_argument('--local_crops_scale', type=float, nargs='+', default=(0.05, 0.4),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for small local view cropping of multi-crop.""")

    # Misc
    parser.add_argument('--data_path', default='/content/drive/MyDrive/datasets/CRC/train/', type=str,
        help='Please specify path to the ImageNet training data.')
    parser.add_argument('--output_dir', default=".", type=str, help='Path to save logs and checkpoints.')
    parser.add_argument('--saveckp_freq', default=2, type=int, help='Save checkpoint every x epochs.')
    parser.add_argument('--seed', default=0, type=int, help='Random seed.')
    parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    # parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")

    # parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    # parser.add_argument('--arch', default='vit_base', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    # parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    # parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set updistributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")




    return parser
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,)+(index,))
        return tuple_with_path


def train_dino(args):
    utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = DataAugmentationDINO(
        args.global_crops_scale,
        args.local_crops_scale,
        args.local_crops_number,
    )
    # dataset = datasets.ImageFolder(args.data_path, transform=transform)
    dataset = ImageFolderWithPaths(args.data_path, transform=transform)
    

    #################################################
    transformKNN = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    data_path="/content/drive/MyDrive/datasets/CRC/test"
    data_path_train="/content/drive/MyDrive/datasets/CRC/train"

    class_name, train_img_list = get_data_list(data_path,0.0)
    train_img_list=sorted(train_img_list)
    # dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transformKNN)

    datasetKNN = ImageFolderWithPaths(data_path, transform=transformKNN)
    datasetKNNTrain = ImageFolderWithPaths(data_path_train, transform=transformKNN)



    # samplerKNN = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    # data_loader_train = torch.utils.data.DataLoader(
    #     dataset_train,
    #     # train_data_set,
    #     # sampler=sampler,
    #     # sampler=train_sample,
    #     batch_size=1,
    #     num_workers=args.num_workers,
    #     pin_memory=True,
    #     drop_last=False,
    # )
        # ============ building network ... ============
    # if "vit" in args.arch:
    #     knnModel = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
    #     print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    # elif "xcit" in args.arch:
    #     knnModel = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    # elif args.arch in torchvision_models.__dict__.keys():
    #     knnModel = torchvision_models.__dict__[args.arch](num_classes=0)
    # else:
    #     print(f"Architecture {args.arch} non supported")
    #     sys.exit(1)
    # if args.use_cuda:
    #     knnModel.cuda()
    # knnModel.eval()

    # # load pretrained weights
    # if os.path.isfile(args.pretrained_weights):
    #     state_dict = torch.load(args.pretrained_weights, map_location="cpu")
    #     if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
    #         print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
    #         state_dict = state_dict[args.checkpoint_key]
    #     # remove `module.` prefix
    #     state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    #     # remove `backbone.` prefix induced by multicrop wrapper
    #     state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
    #     msg = knnModel.load_state_dict(state_dict, strict=False)
    #     print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    # elif args.arch == "vit_small" and args.patch_size == 16:
    #     print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
    #     knnModel.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    # else:
    #     print("Warning: We use random weights.")

    ############################################################################

    









    
    # class_name, train_img_list = get_data_list2('/content/drive/MyDrive/dataSets/CRC/train/',0.0)
    # class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final\\train\\final\\final',0.0)
    # dataset=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    sampler = torch.utils.data.DistributedSampler(dataset, shuffle=True)

    samplerkNN2 = torch.utils.data.DistributedSampler(datasetKNN, shuffle=False)
    samplerkNNTrain = torch.utils.data.DistributedSampler(datasetKNNTrain, shuffle=False)

    
    data_loader = torch.utils.data.DataLoader(
        dataset,
        sampler=sampler,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    data_loaderKNN = torch.utils.data.DataLoader(
        datasetKNN,
        sampler=samplerkNN2,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
        shuffle=False
        
    )


    data_loaderKNNTrain = torch.utils.data.DataLoader(
        datasetKNNTrain,
        sampler=samplerkNNTrain,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True,
        shuffle=False
        
    )

    
    # print("&&&&&")
    # print(data_loaderKNN.dataset.imgs[0])
    print(f"Data loaded: there are {len(dataset)} images.")
    # return data_loader
    # ============ building student and teacher networks ... ============
    # we changed the name DeiT-S for ViT-S to avoid confusions
    # we changed the name DeiT-S for ViT-S to avoid confusions
    args.arch = args.arch.replace("deit", "vit")
    # if the network is a Vision Transformer (i.e. vit_tiny, vit_small, vit_base)
    if args.arch in vits.__dict__.keys():
        student = vits.__dict__[args.arch](
            patch_size=args.patch_size,
            drop_path_rate=args.drop_path_rate,  # stochastic depth
        )
        teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
        embed_dim = student.embed_dim
    # if the network is a XCiT
    elif args.arch in torch.hub.list("facebookresearch/xcit:main"):
        student = torch.hub.load('facebookresearch/xcit:main', args.arch,
                                 pretrained=False, drop_path_rate=args.drop_path_rate)
        teacher = torch.hub.load('facebookresearch/xcit:main', args.arch, pretrained=False)
        embed_dim = student.embed_dim
    # otherwise, we check if the architecture is in torchvision models
    elif args.arch in torchvision_models.__dict__.keys():
        student = torchvision_models.__dict__[args.arch]()
        teacher = torchvision_models.__dict__[args.arch]()
        embed_dim = student.fc.weight.shape[1]
    else:
        print(f"Unknow architecture: {args.arch}")

    # multi-crop wrapper handles forward with inputs of different resolutions
    student = utils.MultiCropWrapper(student, DINOHead(
        embed_dim,
        args.out_dim,
        use_bn=args.use_bn_in_head,
        norm_last_layer=args.norm_last_layer,
    ))
    teacher = utils.MultiCropWrapper(
        teacher,
        DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
    )
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    # synchronize batch norms (if any)
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student = nn.parallel.DistributedDataParallel(student, device_ids=[args.gpu])
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.module.state_dict())
    # there is no backpropagation through the teacher, so no need for gradients
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

    # ============ preparing optimizer ... ============
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    # ============ init schedulers ... ============
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in range(start_epoch, args.epochs):
        data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)

        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")

        ################################
        # torch.save(data_loaderKNN.dataset.img_list, 'img_namesAll.pt')
        train_features,train_label = extract_features(teacher, data_loaderKNN, args.use_cuda, multiscale=args.multiscale)
        # query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
        torch.save(train_features, 'train_featuresAll.pt')


        train_features = nn.functional.normalize(train_features, dim=1, p=2)

        sim = torch.mm(train_features, train_features.T)

        del train_features
        for i in range(len(data_loaderKNN.dataset.imgs)):
          for j in range(len(data_loaderKNN.dataset.imgs)):
              index1=data_loaderKNN.dataset.imgs[i][0].rfind('/')
              index2=data_loaderKNN.dataset.imgs[j][0].rfind('/')
              if data_loaderKNN.dataset.imgs[i][0][index1+4:index1+7] == data_loaderKNN.dataset.imgs[j][0][index2+4:index2+7] and i!=j:
                  sim[i][j]=0
            
          
        simNumpy=sim.cpu().numpy()
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()

        # for i in range(300):
        #     sim[i][i]=-100.0

        # ranks2 = torch.argsort(-sim, dim=0).cpu().numpy()


        count = 0

        for i in range(1000):
            if ranks[1,i]<1000:
                count +=1
        print(count)        
        for i in range(1000,2000):
            if ranks[1,i]>1000 and ranks[1,i]<2000 :
                count +=1
                
        print(count)           
        for i in range(2000,3000):
            if ranks[1,i]>2000 and ranks[1,i]<3000 :
                count +=1
        print(count)  

        precent=count/3000
        print(precent)

        del sim
        del simNumpy
        del ranks



        
        train_features,train_label = extract_features(teacher, data_loaderKNNTrain, args.use_cuda, multiscale=args.multiscale)
        train_features=train_features.cpu()
        sim = torch.mm(train_features[:10000], train_features[:10000].T)

        for i in mask:
          # print(i)
          sim[i[0]][i[1]]=0
          sim[i[1]][i[0]]=0
        KNN=[]
        for i in range(len(sim)):
          # print(i)
          KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:3])
        del sim
        sim = torch.mm(train_features[10000:20000], train_features[10000:20000].T)

        for i in mask:
          # print(i)
          sim[i[0]][i[1]]=0
          sim[i[1]][i[0]]=0
        # KNN=[]
        for i in range(len(sim)):
          # print(i)
          KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:3])
        save('/content/drive/MyDrive/models/dino-main/KNN22.npy', KNN)
        del sim
        
        sim = torch.mm(train_features[:10000], train_features[:10000].T)

        for i in mask:
          # print(i)
          sim[i[0]][i[1]]=0
          sim[i[1]][i[0]]=0
        # KNN=[]
        for i in range(len(sim)):
          # print(i)
          KNN.append(torch.argsort(-sim[:,i], dim=0).cpu().numpy()[:3])
        del sim

        #################################

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))
    








def train_one_epoch(student, teacher, teacher_without_ddp, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,epoch,
                    fp16_scaler, args):
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}/{}]'.format(epoch, args.epochs)
    for it, (images, _,paths,indces) in enumerate(metric_logger.log_every(data_loader, 10, header)):
        # update weight decay and learning rate according to their schedule
        # print('DDD=>',dd)
        # print('images=>',paths)
        # print(index)
        # print(images[0])
        # for index in indces:
        #   temp=[]
        #   for 
        images2,b,c,d=data_loader.dataset.__getitem__(KNN[indces[0]][1])
        for i in range(len(images2)):
          images2[i]=images2[i].unsqueeze(0)

        nearst=[]
        for i in range(1,len(indces)):
          a,b,c,d=data_loader.dataset.__getitem__(KNN[indces[i]][1])
          nearst.append(a)
        
        for i in range(len(images2)):
          for j in range(len(nearst)):
            images2[i]=torch.cat((images2[i], nearst[j][i].unsqueeze(0)), 0)
            # print(images2[i].shape)
        # images2=[torch.Tensor() for _ in range(len(images))]
        # for i in range(len(images2)):
        #   for index in range(1,len(indces)):
        #     a,b,c,d=data_loader.dataset.__getitem__(index)
        #     images2[i]=torch.cat((images2[i], a[i].unsqueeze(0)), 0)
        #     print(images2[i].shape)

        # print(len(images2))
        # print(len(images2[0]))
        # print(images[0].shape)
        it = len(data_loader) * epoch + it  # global training iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]

        # move images to gpu
        images = [im.cuda(non_blocking=True) for im in images]
        images2 = [im.cuda(non_blocking=True) for im in images2]
        
        # teacher and student forward passes + compute dino loss
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            teacher_output = teacher(images[:2])  # only the 2 global views pass through the teacher
            student_output = student(images2)
            loss = dino_loss(student_output, teacher_output, epoch)

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)
        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            optimizer.step()
        else:
            fp16_scaler.scale(loss).backward()
            if args.clip_grad:
                fp16_scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            fp16_scaler.step(optimizer)
            fp16_scaler.update()

        # EMA update for the teacher
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            for param_q, param_k in zip(student.module.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)

        # logging
        torch.cuda.synchronize()
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
        dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output) * dist.get_world_size())

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)


class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(0.1),
            utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops


if __name__ == '__main__':
    parser = argparse.ArgumentParser('DINO', parents=[get_args_parser()])
    args = parser.parse_args("")
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    data_loader=train_dino(args)

Will run the code on one GPU.
| distributed init (rank 0): env://


Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


git:
  sha: N/A, status: clean, branch: N/A

arch: vit_small
batch_size_per_gpu: 8
checkpoint_key: teacher
clip_grad: 3.0
data_path: /content/drive/MyDrive/datasets/CRC/train/
dataset: roxford5k
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
imsize: 224
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 1
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
multiscale: False
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: .
patch_size: 8
pretrained_weights: checkpoint.pth
rank: 0
saveckp_freq: 2
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_cuda: True
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Data loaded: there are 42885 images.
Student and Teacher are built: they are both vit_small network.
Loss, optimizer and schedulers ready.
Found checkpoint at ./checkpoint.pth
=> loaded 'student' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
=> loaded 'teacher' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
=> loaded 'optimizer' from checkpoint: './checkpoint.pth'
=> loaded 'fp16_scaler' from checkpoint: './checkpoint.pth'
=> loaded 'dino_loss' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
Starting DINO training !
Epoch: [32/100]  [   0/5360]  eta: 10:09:01  loss: 2.294613 (2.294613)  lr: 0.000014 (0.000014)  wd: 0.123551 (0.123551)  time: 6.817468  data: 2.410334  max mem: 9367
Epoch: [32/100]  [  10/5360]  eta: 2:43:57  loss: 2.488318 (2.421155)  lr: 0.000014 (0.000014)  wd: 0.123556 (0.123556)  time: 1.838740  data: 0.219431  max mem: 9526
Epoch: [32/100]  [  20/5360]  eta: 2:55:10  loss: 2.2788

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,)+(index,))
        return tuple_with_path
